## Titanic Problem:
#### Learning linear regression with tensor flow and using it to predict whether a passenger of the titanic will survive

Initializing the libraries

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import feature_column as fc
import tensorflow as tf

Getting the Data from csv files into Pandas DataFrame. We need train data and test data, to make sure our models are correct. The survival data is extracted to different DataFrames.

In [13]:
dftrain = pd.read_csv('Dataset/train.csv') # training data
dfeval = pd.read_csv('Dataset/eval.csv') # testing data

y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

Just plotting the data for me to see.

In [ ]:
dftrain.sex.value_counts().plot(kind='barh')

Our data has catagorical data like town or deck and others. Computers don't care about catagorical data. So we need to convert it to numerical data, as computers only care if there is a difference between the data. Feature columns are what we use to send the data to tensor. Here we create two for loops to add the numerical and catagorical data to our feature columns.

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [18]:
print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def